In [25]:
// general includes
#include <iostream>   // std::cout|endl 
#include <vector>   // std::vector
#include <list>   // std::list
#include <utility>   // std::pair
// using namespace std;

## Quick overall recap: 
L1/L2

- *Variables* are defined with a *name* and a **type** (value type, lref/rref type, pointer type).
- All *expressions* have a **type** and a **value category** (lvalue, rvalue).
- References are initialized by **binding** it to an expression.

L3

- *Function names* are **overloaded** (using the sequence parameter types).
- *Function calls* need to be able to uniquely resolved to a specific overload.
- *Move semantics* rely on the overloads for lref/rref types.
- The memory footprint of a *class* is defined by its non-static **data members**.

L4

- Functions directly associated with an instance of a class instance (`this`) are called non-static **member functions**.
- The semantics of a class are defined by its **special member functions** (copy/move-ctor, copy/move-assign, dtor).
- Rule of 0/3/5

L5 

- *Function templates* enable **generic functions** (also member functions)
- **template parameters** can be deduced from the expressions passed to a function call.
- **forwarding-references** + **parameter packs** enable generic forwarding/wrapping by capture the value category of each parameter.

L6 

- *Class templates* enable **generic classes** (i.e. generic data members and member functions)
- **template parameters** can be deduced from the expressions passed to a constructor calls.



## Goals for today
- ...

# Iterators

Using iterators (not only iterators from the standard library, but any iterator-like concept) to operate on data stored in a data structure (i.e. a container) abstracts from details of the implementation:

If all usage scenarios are covered by appropriate iterators, acquiring detailed knowledge of the underlying data structure/container implementation is not required.

**Is the iterator approach a good idea?**
- using iterators is "more safe" than indexing
- if I have "no clue" about the 'length' of a data structure, how can I use indices? Iterator can help with `begin()` and `end()`, knowing an explicit length upfront might not be required for many algorithms
- iterators are "required" if you want to use stdlib algorithms

**Assume you have no knowledge of the implementation of `std::vector` beside that it uses a contiguous memory block which is accessible through `vec.data()` in form of a raw pointer and that it has a `vec.size()` member. How to iterate over all values in the vector and modify the values?**
- w/o iterator:  
  ```cpp
  for (decltype(vec.size()) i = 0; i<vec.size(); ++i) 
    *(vec.data()+i) = ...;
  ```
- using iterators (and range-based for): 
  ```cpp
  for (auto& value: vec)
     value = ...;
  ```

**Assume you have no knowledge of implementation details of `std::list`, but you work with an interface returning an `std::list` object which contains the collection of values you want to work with: how to iterate over all values?**
- w/o iterators: not possible, `std::list` does not expose functionality to interfere with implementation details (e.g. accessing a "node" or moving from node to node using something like `node->next`)
- using iterators: 
  ```cpp
  for (auto& value: list)
     value = ...;
  ```
- the pattern becomes clear: iterators decouple the underlying data structure details from its use

**Examples for interfaces with iterators (not stdlib)?**
- efficient access of dense data [vtk](https://vtk.org/doc/nightly/html/classvtkCellIterator.html#details)
- efficient access of sparse data, e.g. [OpenVDB](https://www.openvdb.org/documentation/doxygen/codeExamples.html#sIteration)
- big advantage: "guaranteed performance" -> implementer knows how to best iterate over the collection

Let's look at a simple example:

In [26]:
struct Widget {
  std::vector<double> data;
  std::vector<bool> active;
  Widget(std::size_t size) : data(size, 0.0), active(size, false) {}
  void set_value(std::size_t idx, double value) { data[idx] = value; }
  void set_status(std::size_t idx, bool status) { active[idx] = status; }
};

This `Widget` can be considered a data structure which holds values together with a flag indicating "activity" via a Boolean status for each value.
Let's assume the only usage scenario is to sweep over all active values. This could look like this:

In [27]:
Widget w(10);
w.set_value(5, 60.0);
w.set_status(5, true);

// iterate over 'active' values in Widget
for (std::size_t i = 0; i < w.data.size(); ++i) {
    if (w.active[i]) {
    // read/write using w.data[i]
    }
}

Although a simple example, it reveals that the user is required to digest the details of the implementation of `Widget`. Let's assume a slight modification resulting in `Widget2`:

In [28]:
struct Widget2 {
  std::vector<std::pair<bool, double>> data;
  Widget2(std::size_t size) : data(size, {0.0, false}) {}
  void set_value(std::size_t idx, double value) { data[idx].second = value; }
  void set_status(std::size_t idx, bool status) { data[idx].first = status; }
};

This change in the implementation details requires again to digest the details of the implementation to conduct the same task:

In [29]:
Widget2 w(10);
w.set_value(5, 60.0);
w.set_status(5, true);
// iterate over 'active' values in Widget2
for (std::size_t i = 0; i < w.data.size(); ++i) {
    if (w.data[i].first) {
    // read/write using w.data[i].second
    }
}

If an iterator for this use case "visit all active values" would be available, this could look like this (independent of the implementation details of `Widget`):

```cpp
// iterate over 'active' values in Widget
for (auto iter = w.begin_active(); iter != w.end_active(); ++iter) {
    double &value = *iter; // access via dereferencing
}  
```

**Describe apparent properties of the iterator object above and how this object is obtained!**
- iterator object must support `operator++` and `operator!=`
- iterator object provides access to value using `operator*`
- `begin_active()`  and `end_active()` return iterator objects apparently pointing to first value and "end"
- if `iter == w.end_active()` the for-loop ends and the loop body is not execute: this means "end" does not represent the "last" value, but something like "one after" or "iteration completed"

## Implementation example: iterator for **Widget**

Without thinking too much about iterators defined in the stdlib, let's try to implement a `IteratorActive` for the `Widget` compatible to the snippet above.
The requirements for the `IteratorActive` are:
- start/begin iterator obtainable by-value from `Widget::begin_active()`
- one-past-the-end iterator obtainable by-value from `Widget::end_active()`
- `operator!=` and `operator++` available
- `operator*` results in a reference to the value allowing read/write access

Mapping these requirements to declarations looks like this (when nesting the iterator class inside `Widget`): 

In [30]:
struct Widget3 {
  std::vector<double> data;
  std::vector<bool> active;
  ///...
  struct IteratorActive {
    double &operator*();
    IteratorActive &operator++();
    bool operator!=(const IteratorActive &other);
  };
  IteratorActive begin_active();
  IteratorActive end_active();
};

Finally, an implementation of a lightweight iterator over active values might look like this:

In [31]:
struct Widget4 {
  std::vector<double> data;
  std::vector<bool> active;    
  //...
  struct IteratorActive {
    Widget4 &ref;             // reference to associated object
    std::size_t pos;          // current state
    double &operator*() { return ref.data[pos]; } // wrapping access 
    IteratorActive& operator++() { // increment to next active value
      while (++pos != ref.data.size()) {
        if (ref.active[pos])
          break;
      }
      return *this;
    }
    bool operator!=(const IteratorActive &other) { // wrapping compare
      return this->pos != other.pos;
    }
  };
  IteratorActive begin_active() {
    IteratorActive iter = {*this, 0};
    return iter.ref.active[iter.pos] ? iter : ++iter; // increment if first value is not active
  }
  IteratorActive end_active() {
    return IteratorActive{*this, data.size()}; // return iterator in state "end"
  }
};

In [32]:
Widget4 w4 = {std::vector<double> {1,2,3,4}, std::vector<bool> {false, true, false, true}};
for (auto iter = w4.begin_active(); iter != w4.end_active(); ++iter) {
    double &value = *iter; // access via dereferencing
    std::printf("active value: %lf\n", value);
}  

active value: 2.000000
active value: 4.000000


## Iterator invalidation 

Let's append an entry (status and value) to a `Widget` from above and observe the effects on iterators constructed before this appending operation:

In [33]:
Widget4 w4 = {std::vector<double> {1,2,3,4}, std::vector<bool> {false, true, false, true}};

auto start = w4.begin_active();
auto end = w4.end_active();
w4.data.push_back(5);
w4.active.push_back(true);

for (auto iter = start; iter != end; ++iter) {
    double &value = *iter; // access via dereferencing
    std::printf("value: %lf\n", value);
}  

for (auto iter = w4.begin_active(); iter != w4.end_active(); ++iter) {
    double &value = *iter; // access via dereferencing
    std::printf("2 value: %lf\n", value);
}  


value: 2.000000
value: 4.000000
2 value: 2.000000
2 value: 4.000000
2 value: 5.000000


It is apparent that `push_back` changes the size of the dynamic memory owned by `data` and `active`.

**Do such "push backs" have consequences for `IteratorActive` instances associated with a `Widget` object?**
- yes, existing iterators associated with an object which is "pushed back" are not updated w.r.t. the changed range of values (i.e. note that `data.size()` is used when construction an `IteratorActive`).
- in general: it depends on implementation details (of the iterator and data structure) which operations on a data structure lead to invalidation of existing iterators.

## Range-based for loop

Above we have implemented an iterator for the special use case "visit all active values in a `Widget`".
Let's resort to a simpler class `WidgetArray` which only holds a single field to explore requirements for a class object to be compatible with a *range-based* `for`-loop [(cppref)](https://en.cppreference.com/w/cpp/language/range-for):


In [34]:
template <typename T, int N> 
struct WidgetArray {
  T data[N];
};

In [35]:
// usage in range-based for loop
WidgetArray<double,10> array;
for (auto &item : array){
  // statements using named variable item;
  item = 7; // no dereferencing required
}

input_line_51:4:17: error: invalid range expression of type '__cling_N530::WidgetArray<double, 10>'; no viable 'begin' function available
for (auto &item : array){
                ^ ~~~~~


Interpreter Error: 

A range-based `for`-loop has the following pattern:

```cpp
for (declaration : object_with_begin_end) {
    // statements using name variable from declaration
}
```

is transformed by the compiler to
```cpp
{
  auto &&range = object_with_begin_end;
  auto begin = range.begin(); // fallback is 'begin(object_wo_begin_end)' 
  auto end = range.end();     // fallback is 'end(object_wo_begin_end)' 
  for ( ; begin != end; ++begin) {
    declaration = *begin;
    // statements using name variable from declaration
  }
}
```

This reveals the requirements:
- the object is required to have member functions `begin` and `end` (fallback are free functions `begin` and `end`)
- the return type of `begin` and `end` has to support `operator++` and `operator!=` 
- additionally, `operator*` is compatible with the declaration of the named variable 

If we would like to support *range-based* `for`-loop to access `WidgetArray` one option is to extend the class by appropriate `begin` and `end` member functions:

In [ ]:

template <typename T, int N> 
struct WidgetArray {
  T data[N]; // raw array -> pointers support all we need already (pointer arithmetic)
  T* begin() { return data; } // iterator is T* -> pointer support ++ != deref.
  T* end() { return data + N; } // as 'end' simply 'the address one past the end' is used
};


Alternatively, free functions can be used, too:

In [ ]:
template <typename T, int N> 
T* begin(WidgetArray<T, N> &array) {
  return array.data;
}
template <typename T, int N> 
T* end(WidgetArray<T, N> &array) {
  return array.data + N;
}

Either way, `WidgetArray` can now be used in a range based for loop;

In [ ]:
WidgetArray<double,4> wa = {{1,2,3,4}};

for (const auto& item: wa)
  std::printf("%lf\n",item);

This is already sufficient for `WidgetArray` as a pointer is used directly as iterator [(cppref:end)](https://en.cppreference.com/w/cpp/iterator/end): raw pointers fulfill the requirements for `operator!=`, `operator++` (pointer arithmetic), and `operator*` (dereferencing).

**Note**: Observing the above, iterators can be thought of as a "generalization of pointer arithmetic" for situations where the underlying implementation does not map to a plain array or other logic ("visit only active values") is desired.

**How would we have to adopt `Widget` and `IteratorActive` above to also work with a range-based for-loop?**
- only the member function names to obtain the begin and end iterators need to be adapted 
  - `begin_active` -> `begin` 
  - `end_active` -> `end`

## Iterators in the standard library, containers, and algorithms 

The standard library uses iterators extensively: all containers support iterators.
Depending of the typically expected underlying implementation for each container in the standard library, a specific category of iterator is supported [(cppref)](https://en.cppreference.com/w/cpp/iterator):

- random-access-iterator [(cppref)](https://en.cppreference.com/w/cpp/named_req/RandomAccessIterator) (`std::array`, `std::vector`, `std::deque`)
  ```cpp
  auto iter = iter + std::rand() % vec.size(); // random access 
  *iter = 5; // writing 
  value = *iter; // reading 
  ++iter; // inc
  --iter; // dec
  iter == iter2; iter != iter2; // compare
  iter = iter2 // assignment/copy does result in two "usable" iterators
  ```

- bidirectional iterator [(cppref)](https://en.cppreference.com/w/cpp/named_req/BidirectionalIterator) (`std::list`, `std::set` (const iter), `std::map` (const iter))
  ```cpp
  *iter = 5; // writing 
  value = *iter; // reading 
  ++iter; // inc
  --iter; // dec
  iter == iter2; iter != iter2; // compare 
  iter = iter2 // assignment/copy does result in two "usable" iterators     
  ``` 
- forward iterator [(cppref)](https://en.cppreference.com/w/cpp/named_req/ForwardIterator) (`std::forward_list`, `std::unordered_set` (const iter), `std::unordered_map` (const iter))
  ```cpp
  *iter = 5; // writing 
  value = *iter; // reading 
  ++iter; // inc
  iter == iter2; iter != iter2; // compare 
  iter = iter2 // assignment/copy does result in two "usable" iterators     
  ``` 

- input iterators [(cppref)](https://en.cppreference.com/w/cpp/named_req/InputIterator)
  ```cpp
  ++iter; // inc
  value = *iter; // reading 
  iter == iter2; iter != iter2; // compare  
  iter = iter2 
  // assignment/copy does is allowed but:
  // but only one of the two can be used further (the other then becomes invalid)
  ```

- output iterators [(cppref)](https://en.cppreference.com/w/cpp/named_req/OutputIterator)
  ```cpp
  ++iter; // inc
  *iter = value; // writing
  // assignment/copy does is allowed but:
  // but only one of the two can be used further (the other then becomes invalid)    
  ```    

**Does it make sense for a container to implement multiple categories of this hierarchy?**
- no, the hierarchy is inclusive, e.g.:
  - a random access iterator *is* also any of the other categories,
  - a forward iterator *is* a output iterator.
  - a forward iterator *is* a input iterator 
  - ...
- only between input and out iterators this inclusion is "broken": 
  - a input iterator *is not* a output operator 
  - a output iterator *is not* a input operator 

### Const iterators

Const iterators (iterators over const values) are available for all containers in the stdlib using `cbegin` and `cend`

In [ ]:
std::vector<int> vec(100,1);
const auto iter1 = vec.begin(); // const iterator object
// ++iter1;    // error
*iter1 = 7; // ok
auto iter2 = vec.cbegin();      //  iterator over const values
++iter2;    // ok
// *iter2 = 7; // error

### Algorithms

The algorithms in the standard library base their interfaces on iterators.
Always the lowest category in the hierarchy of iterator categories which is sufficient for the algorithms is selected for the interface.
This means not all containers support all algorithms.
Let's see a list of examples:

- `std::random_shuffle`, `std::sort` -> random access iterators; [(cppref:random_shuffle)](https://en.cppreference.com/w/cpp/algorithm/random_shuffle)
- `std::reverse_copy`, `std::next_permutation` -> bidirectional iterators [(cppref:reverse_copy)](https://en.cppreference.com/w/cpp/algorithm/reverse_copy)
- `std::fill`, `std::replace`,  `std::search` `std::find`  -> forward iterators [(cppref:find)](https://en.cppreference.com/w/cpp/algorithm/find)
- `std::transform` -> input and output iterators [(cppref:transform)](https://en.cppreference.com/w/cpp/algorithm/transform)
- `std::for_each` -> input iterators [(cppref:for_each)](https://en.cppreference.com/w/cpp/algorithm/for_each)
- `std::fill_n` -> output iterator [(cppref:fill_n)](https://en.cppreference.com/w/cpp/algorithm/fill_n)

The documentations linked above also contain a "possible implementation" which can be used to reason about the iterator category which is required for an algorithm, e.g. for `std::fill_n`:

```cpp
/* from https://en.cppreference.com/w/cpp/algorithm/fill_n */
template<class OutputIt, class Size, class T>
OutputIt fill_n(OutputIt first, Size count, const T& value)
{
    for (Size i = 0; i < count; i++) {
        *first++ = value; 
    }
    return first; 
}
```


Can I implement standard library compatible iterators for my class/container?
- yes, definitely.
- to integrate with the standard library (e.g. algorithms) it is required to define an iterator category (and some more nested types following this scheme:
  ```cpp
  struct CompatibleForwardIterator {
    using iterator_category = std::forward_iterator_tag;
    using value_type = int;
    using difference_type = int;
    using pointer = int*;
    using reference = int&;
    ...
  }
  ```
- for compatibility, the implementation is required to implement all requirements for the selected *iterator category*, e.g. for a [forward iterator](https://en.cppreference.com/w/cpp/named_req/ForwardIterator).

# Summary
- ...
- ...